In [1]:
import numpy as np
from biophi.humanization.methods.sapiens.predict import sapiens_predict_seq, sapiens_predict_chain
import pandas as pd
import numpy as np
import torch
from abnumber import Chain

In [52]:
def get_embeds(chain):
    pred, extra = sapiens_predict_seq(
        seq=chain.seq[:142],
        chain_type=chain.chain_type,
        return_all_hiddens=True
    )
    return extra

In [53]:
def embed_seq(seq, layer=4):
    #if len(seq) > 142:
    #    seq = seq[:142]
    if seq == "QVQLQESGPRLVKPSDTLSLTCTVSGGSITSDSHYWGWVRQSPGKGLEWIASTSDSGSTYHNRPIYYNPSLKSRVTISVATSKNQFSLKLRSVTAADTAMYYCVRQWRYSSGSLDCWGQGILVTVSS" or seq == "EPQLQESGPTLVEASETLSLTCAVSGDSTAACNSFWGWVRQPPGKGLEWVGSLSHCASYWNRGWTYHNPSLKSRLTLALDTPKNLVFLKLNSVTAADTATYYCARFGGEVLRYTDWPKPAWVDLWGRGTLVTVSS":
        pred, extra = sapiens_predict_seq(seq, chain_type="H", model_version='latest', return_all_hiddens=True)
    else:
        ch = Chain(seq, "imgt")
        extra = get_embeds(ch)
    layer_state = extra['inner_states'][layer]
    #layer_reduced = torch.mean(fourth, (0, 1))
    
    return layer_state

In [7]:
chen_data = pd.read_csv("../diplomka/di-pred/data/chen/deduplicated/chen_data.csv", index_col=0)
chen_data.head()

,Antibody_ID,heavy,light,Y
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0


In [5]:
aa_embeddings = {}

In [74]:
for seq in chen_data["heavy"]:
    embeds = embed_seq(seq, 4).numpy()
    for i, aa in enumerate(seq[1:-1]):
        if i > 142:
            break
        this_aa = aa_embeddings.get(aa, [])
        try:
            this_aa.append(embeds[i+1][0])
            aa_embeddings[aa] = this_aa
        except IndexError as e:
            print(seq)
            print(e)
            break

EVQVVESGGGVVQPGRSLRLSCTASGFTFSNFAMGWVRQAPGKGLEWVAFISSDGSNKNYGDSVKGRFTISRDNSKNTVFLQMNSLRVEDTALYYCAKDVGDYKSDEWGT
index 108 is out of bounds for axis 0 with size 108


In [77]:
len(aa_embeddings["G"])

19623

In [83]:
avg_embeddings = {}
for aa, embeds in aa_embeddings.items():
    avg_embeddings[aa] = np.mean(embeds, axis=0)

In [84]:
avg_embeddings["G"]

array([-0.05875249,  0.26576903,  0.0694622 , -0.04194905,  0.24161631,
        0.225295  ,  0.13105331, -0.6727761 , -0.28621334,  0.06681649,
       -0.10881362, -0.10429356,  0.03664941, -0.1732501 ,  0.04213296,
        0.15052891, -0.3503338 ,  0.15385324, -0.08361631,  0.41532037,
        0.1271534 , -0.09759319, -0.05075373, -0.25410998, -0.02562373,
       -0.15043558, -0.4387302 , -0.04847994,  0.05231755,  0.10069021,
       -0.4691429 , -0.13991734, -0.21416298,  0.418264  ,  0.03282569,
        0.439319  , -0.16062318,  0.10267996,  0.66527367, -0.40021572,
        0.3700225 ,  0.25961298, -0.01539211,  0.12580693, -0.5059533 ,
        0.12072579, -0.36542618, -0.01035052,  0.04767674, -0.66159546,
        0.05294256, -0.15926789,  0.01952326, -0.37280717, -0.10008378,
        0.4121887 ,  0.04799857, -0.12909476,  0.0144723 , -0.21966751,
       -0.66507626, -0.5529061 ,  0.07887878, -0.06207951,  0.90561277,
        0.25843486,  0.03269519, -0.28554687, -0.08815382,  0.03

In [85]:
del(aa_embeddings)

In [86]:
for aa, embeds in avg_embeddings.items():
    np.savetxt(f"../diplomka/di-pred/data/aa_embeds/{aa}.txt", embeds)